In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))


True
1
0
NVIDIA GeForce RTX 4060 Laptop GPU


In [5]:
import torch
import torchtext

print(torch.__version__)
print(torchtext.__version__)


OSError: [WinError 127] Не найдена указанная процедура

In [3]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# Загрузка данных
data = pd.read_csv('D:\\Users\\Legion\\datasets\\emotions.csv')
texts = data['text'].values
labels = data['label'].values

# Токенизация и построение словаря
tokenizer = get_tokenizer("basic_english")
vocab = build_vocab_from_iterator(map(tokenizer, texts), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])  # Для неизвестных токенов

# Преобразование текста в числовой вид
text_pipeline = lambda x: vocab(tokenizer(x))
tokenized_texts = [text_pipeline(text) for text in texts]

# Паддинг последовательностей
tokenized_texts = [torch.tensor(seq, dtype=torch.long) for seq in tokenized_texts]
padded_texts = pad_sequence(tokenized_texts, batch_first=True)
labels = torch.tensor(labels, dtype=torch.long)

# Создание даталоадера
dataset = TensorDataset(padded_texts, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


OSError: [WinError 127] Не найдена указанная процедура

In [5]:
import torch.nn as nn
import nengo
import nengo_dl

# Пример простой архитектуры LMU для последовательностей
class TextLMUModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(TextLMUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lmu = nengo_dl.TensorNode(
            nengo.LMUCell(order=1, theta=100, input_size=embedding_dim, hidden_size=hidden_dim),
            size_in=embedding_dim,
            size_out=hidden_dim,
        )
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lmu(x)
        x = self.fc(x)
        return x

# Параметры модели
vocab_size = len(vocab)
embedding_dim = 64
hidden_dim = 128
output_dim = len(set(labels.tolist()))

model = TextLMUModel(vocab_size, embedding_dim, hidden_dim, output_dim)


KeyError: 'emotion'

In [ ]:
import torch.optim as optim

# Настройка оптимизатора и функции потерь
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Цикл обучения
for epoch in range(10):  # Задайте количество эпох
    total_loss = 0
    for batch_texts, batch_labels in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_texts)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")
